In [3]:
import numpy as np
import pandas as pd
import yfinance as yf

## Returns

In [42]:
""" Overview
one period total return formula = (P_t+1 + dividend - P_t) / (P_t)
1+R format = R is the percent return + 1 to simplify computation
multiple time period = (1+R1) * (1+R2) ... - 1
annualized return: compound growth at annual scale
    ex. monthly return of 1% is (1.01)^12 - 1

"""

prices_a = [8.70, 8.91, 8.71]
returns_a = np.array(prices_a[1:]) / np.array(prices_a[:-1]) - 1
prices= pd.DataFrame({'BLUE':[8.7, 8.91, 8.7, 8.4, 8.6],
                          'ORANGE':[11.9, 12.4, 11.8, 12.4, 12.9] })

# two ways of calculating returns
returns = prices/prices.shift() - 1
returns = prices.pct_change()
print('Returns:\n',returns.tail(3))

# compound returns np and pd versions
compound = np.prod(returns+1) - 1
compound = (returns+1).prod() - 1
print('Compound returns:\n',compound)

# annualization
monthly_ret = .02
annualized_monthly_ret = (1 + monthly_ret)**12 - 1
quarterly_ret = .03
annualized_quarterly_ret = (1 + quarterly_ret)**4 - 1
daily_ret = .001
annualized_daily_ret = (1+daily_ret)**252 - 1

Returns:
        BLUE    ORANGE
2 -0.023569 -0.048387
3 -0.034483  0.050847
4  0.023810  0.040323
Compound returns:
 BLUE     -0.011494
ORANGE    0.084034
dtype: float64


## Risk and volatility

In [25]:
# download sp500, dow30 and russel2000
spy = yf.download('SPY')
dia = yf.download('DIA')
iwm = yf.download('IWM')
qqq = yf.download('QQQ')

prices = pd.concat([dia['Close'], spy['Close'], iwm['Close'], qqq['Close']],axis=1).dropna()
prices.columns = ['dia', 'spy', 'iwm', 'qqq']
prices.tail(3)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,dia,spy,iwm,qqq
Date,,,,
2022-09-30,287.299988,357.179993,164.919998,267.260010
2022-10-03,294.790009,366.609985,169.289993,273.529999
2022-10-04,303.140015,377.970001,176.000000,282.130005


In [28]:
# sharpe ratio: (returns - risk free rate) / volatility 
# measures excess risk adjusted returns
returns = prices.pct_change().dropna()
volatility = returns.std()
print('Volatility\n',volatility)

Volatility
 dia    0.011924
spy    0.012391
iwm    0.015306
qqq    0.016736
dtype: float64


In [57]:
worst_day_per_year = returns.groupby(returns.index.year).min()
yearly_returns = (returns+1).groupby(returns.index.year).prod() - 1
print('Average yearly returns:\n',yearly_returns.mean())
print('Vol of yearly returns:\n',yearly_returns.std())


Average yearly returns:
 dia    0.059340
spy    0.060807
iwm    0.078186
qqq    0.095910
dtype: float64
Vol of yearly returns:
 dia    0.150530
spy    0.177677
iwm    0.193642
qqq    0.279193
dtype: float64


In [45]:
total_returns = (returns+1).prod()
print('Total returns in %:\n',total_returns*100)

Total returns in %:
 dia    293.953954
spy    273.891305
iwm    384.962406
qqq    362.868173
dtype: float64


In [58]:
# annualized monthly vol
last_month = returns.tail(30)
annualized_vol = last_month.std() * np.sqrt(12)
annualized_vol

dia    0.051534
spy    0.057682
iwm    0.065768
qqq    0.065923
dtype: float64

In [59]:
RISK_FREE_RATE = .03
sharpe = (yearly_returns.mean() - RISK_FREE_RATE) / yearly_returns.std()
sharpe

dia    0.194911
spy    0.173387
iwm    0.248841
qqq    0.236072
dtype: float64